In [780]:
import pandas as pd
import os
import csv
import psycopg2
import numpy as np


# Configuración necesaria para que decouple detecte .env (donde están guardadas las configuraciones)
from decouple import config
from decouple import AutoConfig

config = AutoConfig(search_path=os.getcwd())

In [781]:
# Acá simplemente armamos una lista con todas las direcciones enteras de nuestros archivos csv
# Decidí armarlo en una función por si a la hora de utilizarlo se quisiera cambiar la dirección raíz
# Como predeterminado deje datasets que suele ser una dirección raíz común

def directoryLister(rootdir = 'datasets/'):
    filesDirs = []
    
    for rootdir, dirs, files in os.walk(rootdir):
        for file in files:
            filesDirs.append(os.path.join(rootdir, file))
        
    print(filesDirs)
    return filesDirs

In [782]:
# Acá armamos una lista con todos los dataframes provenientes de los csv's

def dfListCreator(fileDirs):
    dataFrames = []
    for j in fileDirs:
        with open(j, 'r') as f:
            #d_reader = csv.DictReader(f)

            #get fieldnames from DictReader object and store in list
            #headers = d_reader.fieldnames
            
            dataFrames.append(pd.read_csv(j))
        f.close()
    return dataFrames
    # I had to use the column names, because of the bug that makes some rows one column larger

In [783]:
def column_checker(df,columns):
    for j in columns:
        if j not in df.columns:
            print('{} no está entre las columnas'.format(j))
            return False

In [784]:
# La función column_checker está diseñada para evitar que un dataset malo corte el proceso y permitir así que los datasets buenos sean procesados
# No la uso en este caso, por que no queda claro que espera Alkemy , y como solo tengo 3 datasets , terminaria perdiendo mucha información
# Para evitar esto hice a mano los casos necesarios. Pero en un caso real, probablemente descartaría los datasets rotos, especialmente si fueran muchos
# Dependiendo siempre de si puedo o no prescindir de la información que estos brindan
# De todas maneras desde .env se puede elegir usar el checker o no.


CHECKER = config('CHECKER')

def columnSelector(dfList,columns, columns_alt = '[]'):
    for j in range(0,len(dfList)):
        dfList[j].columns = dfList[j].columns.str.lower()
        dfList[j].columns = dfList[j].columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

        if CHECKER == True:
            if column_checker(dfList[j],columns):
                dfList[j] = dfList[j][columns]

        else:
            if 'domicilio' in dfList[j].columns:
                dfList[j] = dfList[j][columns]

                dfList[j].rename(columns = {'idprovincia':'id_provincia',
                                            'iddepartamento': 'id_departamento',
                                            'categoria':'categoria',
                                            'cp':'codigo_postal',
                                            'telefono':'numero_de_telefono',}, inplace = True) 

            elif 'direccion' in dfList[j].columns:
                dfList[j] = dfList[j][columns_alt]

                dfList[j].rename(columns = {'idprovincia':'id_provincia',
                                            'iddepartamento': 'id_departamento',
                                            'categoria':'categoria',
                                            'cp':'codigo_postal',
                                            'telefono':'numero_de_telefono',
                                            'direccion':'domicilio'}, inplace = True)

    

def mailCleaner(df_list):
    for df in df_list:
        #df['numero_de_telefono']
        df.loc[df['mail'].isnull() , ['mail']] = 'NULL'
        df.loc[df['mail'] == np.nan , ['mail']] = 'NULL'
        #df['numero_de_telefono'] = df['numero_de_telefono'].astype(float)
        df.loc[df['web'].isnull() , ['web']] = 'NULL'
        df.loc[df['web'] == 's/d' , ['web']] = 'NULL'
        df.loc[df['cod_loc'].isnull() , ['cod_loc']] = 'NULL'
        df.loc[df['id_provincia'].isnull() , ['id_provincia']] = 'NULL'
        df.loc[df['id_departamento'].isnull() , ['id_departamento']] = 'NULL'
        df.loc[df['categoria'].isnull() , ['categoria']] = 'NULL'
        df.loc[df['nombre'].isnull() , ['nombre']] = 'NULL'
        df.loc[df['localidad'].isnull() , ['localidad']] = 'NULL'
        df.loc[df['provincia'].isnull() , ['provincia']] = 'NULL'
        df.loc[df['domicilio'].isnull() , ['domicilio']] = 'NULL'
        df.loc[df['codigo_postal'].isnull() , ['codigo_postal']] = 'NULL'
        df.loc[df['numero_de_telefono'].isnull() , ['numero_de_telefono']] = 'NULL'
        
        df.loc[df['numero_de_telefono'] == 'nan' , ['numero_de_telefono']] = 'NULL'
        


        
        df = df.dropna(how='all')
#def typeConversion(df):
    


In [785]:
# Este pipeline, prepara los tres datasets limpiando las columnas no requeridas, basado en la primera consigna

def dataPipeline():
    
    DATASETS_ROOT_DIR = config('DATASETS_DIR')

    # Obtiene direcciones de mis archivos csv en base a mi rootdir
    fileDirs = directoryLister(DATASETS_ROOT_DIR)
    
    # Crea una lista con los dataframes en base a los csv
    df_list = dfListCreator(fileDirs)

    # Procesa cada dataframe para seleccionar ciertas columnas establecidas en la configuracion .env
    columns = ['cod_loc','idprovincia','iddepartamento','categoria','provincia','localidad','nombre','domicilio','cp','telefono','mail','web']
    columns_alt = ['cod_loc','idprovincia','iddepartamento','categoria','provincia','localidad','nombre','direccion','cp','telefono','mail','web']
    CHECKER = config('CHECKER')

    columnSelector(df_list, columns, columns_alt)

    mailCleaner(df_list)

    return df_list

    

In [786]:
df_list = dataPipeline()
df_list[0].iloc[2]

['datasets/bibliotecas-populares/2022-April/bibliotecas-populares-29-04-2022.csv', 'datasets/museos/2022-April/museos-29-04-2022.csv', 'datasets/salas-de-cine/2022-April/salas-de-cine-29-04-2022.csv']


cod_loc                                      2000010.0
id_provincia                                       2.0
id_departamento                                 2000.0
categoria                        Bibliotecas Populares
provincia              Ciudad Autónoma de Buenos Aires
localidad                       Ciudad de Buenos Aires
nombre                Biblioteca Popular 12 de Octubre
domicilio                         Calle Arengreen 1187
codigo_postal                                 C1405CYM
numero_de_telefono                          49880766.0
mail                    popular12deoctubre@hotmail.com
web                                               NULL
Name: 2, dtype: object

In [787]:
def procesarCines(df):
    # Descarta las columnas innecesarias, despues corrije la columna espacio_INCAA que tiene muchos espacios vacíos
    # Finalmente transforma el tipo a enteros y luego cuenta cada apricion de cada columna , según la provincia
    
    df = df[['Provincia','Butacas','Pantallas', 'espacio_INCAA']]
    
    df.loc[df['espacio_INCAA'] == 'SI' , ['espacio_INCAA']] = 1
    df.loc[df['espacio_INCAA'] == 'si' , ['espacio_INCAA']] = 1
    df.loc[df['espacio_INCAA'] != 1 , ['espacio_INCAA']] = 0
    
    df = df.dropna()
    
    df['espacio_INCAA'] = df['espacio_INCAA'].astype(int)
    df['Butacas'] = df['Butacas'].astype(int)
    df['Pantallas'] = df['Pantallas'].astype(int)

    
    df = df.groupby(['Provincia']).sum()
    df = df.reset_index(level=0)
    df.columns = map(str.lower, df.columns)
    
    #df.set_index('provincia', inplace=True)
    #print(df)
    #a = df.groupby(["Provincia"]).count()['']
    # print(df)
    return df


In [788]:
## Este pipeline prepara el datasets de cines, para ser subido a la base de datos, basado en la tercera consigna

def cinesPipeline():
    cinesDir = directoryLister('datasets/salas-de-cine')
    cinesDf = dfListCreator(cinesDir)
    cinesDf = procesarCines(cinesDf[-1]) # la última fecha
    return cinesDf

In [789]:
df = cinesPipeline()

df

['datasets/salas-de-cine/2022-April/salas-de-cine-29-04-2022.csv']


,provincia,butacas,pantallas,espacio_incaa
0,Buenos Aires,92722,357,19
1,Catamarca,3200,12,1
2,Chaco,2469,14,1
3,Chubut,2682,10,4
4,Ciudad Autónoma de Buenos Aires,31386,153,3
5,Corrientes,3370,17,1
6,Córdoba,20799,105,2
7,Entre Ríos,4086,17,2
8,Formosa,1184,4,1
9,Jujuy,2277,5,2


In [790]:
from importlib import reload
reload(serverLoader)
import serverLoader
serverLoader.createTables()
serverLoader.updateTablesCine(cinesDf)

Tablas creadas


In [791]:
serverLoader.updateTableDatos(df_list[0],'datos_bibliotecas')